In [26]:
import pandas as pd

1. Look at the PBA50+ BAUS zoning for parcels

In [27]:
# 2050 output because as built_dua updates the max, max dua calculation is updated
pz = pd.read_csv("M:/urban_modeling/baus/PBA50Plus/Draft_Blueprint/\
PBA50Plus_Draft_Blueprint_v8_znupd_nodevfix_interimoutput/core_summaries/\
PBA50Plus_Draft_Blueprint_v8_znupd_nodevfix_interimoutput_parcel_summary_2010.csv")

2. Add growth geography variables for understanding parcels' location

In [28]:
# parcels geography input file
pg = pd.read_csv("M:/urban_modeling/baus/BAUS Inputs/basis_inputs/crosswalks/parcels_geography_2024_02_14.csv")

pz = pz.merge(pg[['PARCEL_ID', 'juris', 'gg_id', 'pda_id', 'tra_id', 'hra_id']], 
                  left_on='parcel_id', right_on='PARCEL_ID', how='left')

3. Add the max built DUA/FAR in a parcel's TAZ, used in the BAUS zoning processing

In [29]:
# parcel to TAZ crosswalk
ptaz = pd.read_csv("M:/urban_modeling/baus/BAUS Inputs/basis_inputs/crosswalks/2020_08_17_parcel_to_taz1454sub.csv")
pz = pz.merge(ptaz[['PARCEL_ID', 'ZONE_ID']], left_on='parcel_id', right_on='PARCEL_ID', how='left')

max_dua_builtdua_taz = pz.built_dua.groupby(pz.ZONE_ID).max().fillna(4)
pz['max_dua_builtdua_taz'] = pz['ZONE_ID'].map(max_dua_builtdua_taz).fillna(4)

max_far_builtdua_taz = pz.built_far.groupby(pz.ZONE_ID).max().fillna(4)
pz['max_dua_builtfar_taz'] = pz['ZONE_ID'].map(max_far_builtdua_taz).fillna(4)

4. Add the base zoning to parcels in order to lookup the corresponding zoning input

In [30]:
# latest parcels zoning file
pzon = pd.read_csv("M:/urban_modeling/baus/BAUS Inputs/basis_inputs/zoning/zoning_parcels_2024-04-30.csv")
# latest parcel lookup file
zl = pd.read_csv("M:/urban_modeling/baus/BAUS Inputs/basis_inputs/zoning/zoning_lookup_2024-04-30.csv")
zl = zl.rename(columns={"name": "zoning_name", "max_dua":"max_dua_zoning", "max_far": "max_far_zoning"})

# merge the zoning lookup to parcels
pz = pz.merge(pzon[['PARCEL_ID', 'zoning_id']], left_on='parcel_id', right_on='PARCEL_ID', how='left')\
.merge(zl[['id', 'zoning_name', 'max_dua_zoning', 'max_far_zoning']], left_on='zoning_id', right_on='id', how='left')

5. Inspect the BAUS zonign based on the processing steps

BAUS processing steps / zoning variables in the table:
- **max_dua_zoning** is the input zoning
- **max_dua_imputation** is zoning with imputation and residential allowance applied
- **max_dua_imputation_upzoning** is zoning with imputation and upzoning and residential allowance applied
- **maz_dua_builtfar_taz** is the highest max DUA in the TAZ
- **max_dua** is zoning with imputation and upzoning, residenital allowance, nodev modification, outside of urban area modification, \
and lowest built density modification applied

### DUA

In [31]:
print("Number of parcels with DUA > 40 is {}".format(len(pz.loc[pz.max_dua > 40])))
dua = pz.loc[pz.max_dua > 40][['parcel_id', 'zoning_name', 'max_dua_zoning', 'juris', 'tra_id', 'pda_id',
                               'max_dua_imputation', 'max_dua_imputation_upzoning', 'max_dua_builtdua_taz', 'max_dua']].\
                                sort_values(by='max_dua', ascending=False)

Number of parcels with DUA > 40 is 242955


#### DUA from upzoning

In [32]:
dua_upzoning = dua.loc[(dua.max_dua_imputation_upzoning > dua.max_dua_imputation) & (dua.max_dua_imputation_upzoning == dua.max_dua)]
print("Number of parcels with DUA > 200 from upzoning step is {}".format(len(dua_upzoning.loc[dua_upzoning.max_dua > 200])))
dua_upzoning.loc[dua_upzoning.max_dua > 200].head()

Number of parcels with DUA > 200 from upzoning step is 0


,parcel_id,zoning_name,max_dua_zoning,juris,tra_id,pda_id,max_dua_imputation,max_dua_imputation_upzoning,max_dua_builtdua_taz,max_dua


In [33]:
dua_upzoning_high_nongg = dua_upzoning.loc[(dua_upzoning.max_dua > 200) & (dua_upzoning.tra_id.isnull()) & (dua_upzoning.pda_id.isnull())]
print("Number of parcels with DUA > 200 from upzoning step but not in a TRA or PDA is {}".format(len(dua_upzoning_high_nongg)))
dua_upzoning_high_nongg.head()

Number of parcels with DUA > 200 from upzoning step but not in a TRA or PDA is 0


,parcel_id,zoning_name,max_dua_zoning,juris,tra_id,pda_id,max_dua_imputation,max_dua_imputation_upzoning,max_dua_builtdua_taz,max_dua


#### DUA from Imputation

In [34]:
dua_imputation = dua.loc[(dua.max_dua_imputation > dua.max_dua_zoning) & (dua.max_dua_imputation == dua.max_dua)]
print("Number of parcels with DUA > 40 from imputation is {}".format(len(dua_imputation.loc[dua_imputation.max_dua > 40])))
dua_imputation.loc[dua_imputation.max_dua > 40].head()

Number of parcels with DUA > 40 from imputation is 0


,parcel_id,zoning_name,max_dua_zoning,juris,tra_id,pda_id,max_dua_imputation,max_dua_imputation_upzoning,max_dua_builtdua_taz,max_dua


In [35]:
dua_imputation_high_nongg = dua_imputation.loc[(dua_imputation.max_dua > 40) & (dua_imputation.tra_id.isnull()) & (dua_imputation.pda_id.isnull())]
print("Number of parcels with DUA > 40 from imputation and not in a TRA or PDA is {}".format(len(dua_imputation_high_nongg)))
dua_imputation_high_nongg

Number of parcels with DUA > 40 from imputation and not in a TRA or PDA is 0


,parcel_id,zoning_name,max_dua_zoning,juris,tra_id,pda_id,max_dua_imputation,max_dua_imputation_upzoning,max_dua_builtdua_taz,max_dua


#### DUA from Max TAZ Built DUA

In [36]:
dua_builtdua_taz_max = dua.loc[(dua.max_dua_builtdua_taz == dua.max_dua) & (dua.max_dua_builtdua_taz > dua.max_dua_zoning)]
print("Number of parcels with DUA > 40 set by building to the lowest built DUA {}".format(len(dua_builtdua_taz_max.loc[dua_builtdua_taz_max.max_dua > 40])))
dua_builtdua_taz_max.loc[dua_builtdua_taz_max.max_dua > 40].head()

Number of parcels with DUA > 40 set by building to the lowest built DUA 8166


,parcel_id,zoning_name,max_dua_zoning,juris,tra_id,pda_id,max_dua_imputation,max_dua_imputation_upzoning,max_dua_builtdua_taz,max_dua
239898,1025891,NODEV_pba40,0.0,san_francisco,TRA1,San Francisco - J Church and Mission Corridor,0.0,200.0,156.340091,156.340091
1595926,1041623,RH-1 (HE Data),60.5,san_francisco,TRA1,NaN,60.5,200.0,156.340091,156.340091
1597461,1041324,P (BASIS),0.0,san_francisco,TRA1,NaN,0.0,200.0,156.340091,156.340091
1422527,1041520,RH-1 (HE Data),98.0,san_francisco,TRA1,San Francisco - J Church and Mission Corridor,98.0,200.0,156.340091,156.340091
274324,1024933,RH-1 (HE Data),43.8,san_francisco,TRA1,NaN,43.8,200.0,156.340091,156.340091


In [37]:
dua_builtdua_taz_max_high_nongg = dua_builtdua_taz_max.loc[(dua_builtdua_taz_max.max_dua > 40) & 
                                  (dua_builtdua_taz_max.tra_id.isnull()) & (dua_builtdua_taz_max.pda_id.isnull())]
print("Number of parcels with DUA > 40 from max built dua in TAZ and not in a TRA or PDA is {}".format(len(dua_builtdua_taz_max_high_nongg)))
dua_builtdua_taz_max_high_nongg.head()

Number of parcels with DUA > 40 from max built dua in TAZ and not in a TRA or PDA is 0


,parcel_id,zoning_name,max_dua_zoning,juris,tra_id,pda_id,max_dua_imputation,max_dua_imputation_upzoning,max_dua_builtdua_taz,max_dua


#### DUA from Zoning Input

In [38]:
dua_zoning = dua.loc[(dua.max_dua_zoning == dua.max_dua)]
print("Number of parcels with DUA > 40 from input zoning is {}".format(len(dua_zoning)))
dua_zoning.head()

Number of parcels with DUA > 40 from input zoning is 68313


,parcel_id,zoning_name,max_dua_zoning,juris,tra_id,pda_id,max_dua_imputation,max_dua_imputation_upzoning,max_dua_builtdua_taz,max_dua
1383244,902326,C-3-G (HE Data),895.2,san_francisco,TRA1,San Francisco - Downtown/Van Ness/Northeast Ne...,895.2,895.2,992.821485,895.2
150228,893650,DOWNTOWN- OFFICE-300-S_pba40,871.2,san_francisco,TRA1,San Francisco - Downtown/Van Ness/Northeast Ne...,871.2,871.2,1018.311575,871.2
1819990,893641,DOWNTOWN- OFFICE-300-S_pba40,871.2,san_francisco,TRA1,San Francisco - Downtown/Van Ness/Northeast Ne...,871.2,871.2,1018.311575,871.2
1355772,893647,DOWNTOWN- OFFICE-300-S_pba40,871.2,san_francisco,TRA1,San Francisco - Downtown/Van Ness/Northeast Ne...,871.2,871.2,1018.311575,871.2
1597067,981776,RH DTR (HE Data),749.4,san_francisco,TRA1,San Francisco - Transbay/Rincon Hill,749.4,749.4,927.653103,749.4


In [39]:
dua_zoning_high_nongg = dua_zoning.loc[(dua_zoning.max_dua > 40) & 
                                       ((dua_zoning.tra_id.isnull()) & (dua_zoning.pda_id.isnull()))]
print("Number of parcels with DUA > 40 from input zoning and not in a TRA or PDA is {}".format(len(dua_zoning_high_nongg)))
dua_zoning_high_nongg.head()

Number of parcels with DUA > 40 from input zoning and not in a TRA or PDA is 15997


,parcel_id,zoning_name,max_dua_zoning,juris,tra_id,pda_id,max_dua_imputation,max_dua_imputation_upzoning,max_dua_builtdua_taz,max_dua
1220829,900063,85' Height Allowed (HE Data),243.9,san_francisco,NaN,NaN,243.9,243.9,254.429338,243.9
1178237,907017,85' Height Allowed (HE Data),243.9,san_francisco,NaN,NaN,243.9,243.9,292.793462,243.9
83002,900100,85' Height Allowed (HE Data),243.9,san_francisco,NaN,NaN,243.9,243.9,254.429338,243.9
408213,900476,85' Height Allowed (HE Data),243.9,san_francisco,NaN,NaN,243.9,243.9,345.596402,243.9
1052216,900580,85' Height Allowed (HE Data),243.9,san_francisco,NaN,NaN,243.9,243.9,292.793462,243.9


In [40]:
dua_zoning_high_nongg_nonbig3 = dua_zoning.loc[(dua_zoning.parcel_id.isin(dua_zoning_high_nongg.parcel_id)) & 
            (dua_zoning.juris != 'san_francisco') & (dua_zoning.juris != 'san_jose') & (dua_zoning.juris != 'oakland')]
print("Number of parcels with DUA > 40 from zoning input and not in a TRA, PDA, or Big 3 is {}".format(len(dua_zoning_high_nongg_nonbig3)))
dua_zoning_high_nongg_nonbig3.head()

Number of parcels with DUA > 40 from zoning input and not in a TRA, PDA, or Big 3 is 3280


,parcel_id,zoning_name,max_dua_zoning,juris,tra_id,pda_id,max_dua_imputation,max_dua_imputation_upzoning,max_dua_builtdua_taz,max_dua
689158,525678,RH (HE Data),125.0,concord,NaN,NaN,125.0,125.0,218.328542,125.0
443090,523121,RH (HE Data),125.0,concord,NaN,NaN,125.0,125.0,218.328542,125.0
607876,109967,R-4 (HE Data),124.5,berkeley,NaN,NaN,124.5,124.5,163.524575,124.5
53323,110056,R-4 (HE Data),124.5,berkeley,NaN,NaN,124.5,124.5,163.524575,124.5
888458,117199,R-4 (HE Data),124.5,berkeley,NaN,NaN,124.5,124.5,236.041025,124.5


In [41]:
# export associated zoning codes to check them
#dua_zoning_high_nongg_nonbig3.zoning_name.value_counts().to_csv("high base zoning counts.csv")

In [42]:
# export associated parcels to view location of input zoning on a map (can dissolve on zoning code)
#dua_zoning_high_nongg_nonbig3.to_csv("dua_zoning_high_nongg_nonbig3.csv")

### FAR